In [1]:
import sqlite3
from datetime import datetime
# from db_utils import *
import os
from tqdm import tqdm
import pandas as pd

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 169813)

In [3]:
def get_db_conn():
    try:
        conn = sqlite3.connect('file:rahmanimission.db?mode=rw', uri=True)
        print('Database exists with name: "rahmanimission"')
    except sqlite3.Error as e:
        print(e)
        #print('Database file dosen\'t exists.')
        conn = sqlite3.connect('rahmanimission.db')
        print('New database created named: "rahmanimission"')
        conn.cursor().executescript('''
            CREATE TABLE IF NOT EXISTS track_log
            (last_1 TEXT,last_2 TEXT,last_3 TEXT,last_4 TEXT,last_5 TEXT,last_6 TEXT,last_7 TEXT);

            INSERT INTO track_log (last_1,last_2,last_3,last_4,last_5,last_6,last_7) 
            VALUES ("last_1","last_2","last_3","last_4","last_5","last_6","last_7");
        ''')

    conn.cursor().executescript('''
        CREATE TABLE IF NOT EXISTS Progress
        (id TEXT PRIMARY KEY, email TEXT ,name TEXT ,total_mins_learned INTEGER, skills_leveled_up INTEGER,
        [skill_w/o_progress] INTEGER, teacher_id TEXT, links TEXT, delta TEXT);
        
        CREATE TABLE IF NOT EXISTS Activities
        (student_id Text, activity TEXT, teacher_id TEXT, date TEXT, level TEXT, change TEXT, time NUMERIC,
        correct_problems NUMERIC, total_problems NUMERIC,[correct/total_problems] TEXT);
        
        CREATE TABLE IF NOT EXISTS Assignments
        (assignment Text, student_id TEXT, teacher_id TEXT, due_date TEXT, status TEXT, attempts INTEGER, best_score FLOAT,
        last_1 NUMERIC,last_2 NUMERIC,last_3 NUMERIC,last_4 NUMERIC,last_5 NUMERIC,last_6 NUMERIC,last_7 NUMERIC);
    ''')
    return conn
db_conn = get_db_conn()

Database exists with name: "rahmanimission"


In [4]:
def update_assignment(student_id, df, teacher_id, db_conn):
    # print('Updating Assignments of', student_id,'...')
    db_cur = db_conn.cursor()
    insert_sql = '''INSERT INTO Assignments (assignment, student_id, teacher_id, due_date, status, attempts, best_score)
                VALUES (?, ?, ?, ?, ?, ?, ?)'''
    delete_sql = '''DELETE FROM Assignments WHERE student_id = ? AND assignment = ?'''
    
    db_cur.execute('SELECT * FROM track_log')
    t = db_cur.fetchone()
    query_track = f'''SELECT [{t[0]}], [{t[1]}], [{t[2]}], [{t[3]}], [{t[4]}], [{t[5]}], [{t[6]}] FROM 
                Assignments WHERE student_id = ? AND assignment = ?'''
    update_track = f'''UPDATE Assignments SET [{t[0]}]=?, [{t[1]}]=?, [{t[2]}]=?, [{t[3]}]=?, [{t[4]}]=?, [{t[5]}]=?, [{t[6]}]=? 
                    WHERE student_id = ? AND assignment = ?'''

    len_df = len(df)
    for index in range(len_df):
        db_cur.execute(query_track, (student_id, df.loc[index, 'Assignment']))
        row = db_cur.fetchone()
        db_cur.execute(delete_sql, (student_id, df.loc[index, 'Assignment']))
        attempts = 0
        try:
            attempts = int(df.loc[index, 'Attempts'])
        except:
            pass
        db_cur.execute(insert_sql, (df.loc[index, 'Assignment'], student_id, teacher_id, df.loc[index, 'Due date & time'],
                                    df.loc[index, 'Status'], attempts, df.loc[index, 'Best Score']))
        if row is not None:
            db_cur.execute(update_track, (row[0], row[1], row[2], row[3], row[4], row[5], row[6], 
                                          student_id, df.loc[index, 'Assignment']))

    db_conn.commit()
    # print('Assignments updated of', student_id)

In [5]:
def update_asgn_track(db_conn, track_time=""):
    if track_time == "":
        track_time = datetime.now().strftime('%b %d,%Y at %H:%M')
    db_cur = db_conn.cursor()
    db_cur.execute('SELECT * FROM track_log')
    t = db_cur.fetchone()
    
    if track_time in t:
        print('Aborting Track Update !!!')
        print('You have already updated data for time',track_time)
        print('Try after atleast 60 secs or with different column name...')
        return
    
    db_cur.executescript(f'''UPDATE Assignments SET [{t[6]}] = [{t[5]}];
                            UPDATE Assignments SET [{t[5]}] = [{t[4]}];
                            UPDATE Assignments SET [{t[4]}] = [{t[3]}];
                            UPDATE Assignments SET [{t[3]}] = [{t[2]}];
                            UPDATE Assignments SET [{t[2]}] = [{t[1]}];
                            UPDATE Assignments SET [{t[1]}] = [{t[0]}];
                            UPDATE Assignments SET [{t[0]}] = attempts;''')
    
    db_cur.executescript(f'''ALTER TABLE Assignments RENAME COLUMN [{t[0]}] TO [{track_time}];
                            ALTER TABLE Assignments RENAME COLUMN [{t[1]}] TO [{t[0]}];
                            ALTER TABLE Assignments RENAME COLUMN [{t[2]}] TO [{t[1]}];
                            ALTER TABLE Assignments RENAME COLUMN [{t[3]}] TO [{t[2]}];
                            ALTER TABLE Assignments RENAME COLUMN [{t[4]}] TO [{t[3]}];
                            ALTER TABLE Assignments RENAME COLUMN [{t[5]}] TO [{t[4]}];
                            ALTER TABLE Assignments RENAME COLUMN [{t[6]}] TO [{t[5]}];''')
    db_cur.execute('UPDATE track_log SET last_1= ?, last_2= ?, last_3= ?, last_4= ?, last_5= ?, last_6= ?, last_7= ?',
                   (track_time, t[0], t[1], t[2], t[3], t[4], t[5]))
    db_conn.commit()

In [6]:
def get_pd_table(table_name, db_conn, student_id=""):
    
    if student_id != "":
        if table_name.upper() =='PROGRESS':
            df = pd.read_sql(f'SELECT * FROM {table_name} WHERE id = "{student_id}"', db_conn)
        else:
            df = pd.read_sql(f'SELECT * FROM {table_name} WHERE student_id = "{student_id}"', db_conn)
    else:
        df = pd.read_sql(f'SELECT * FROM {table_name}', db_conn)
        
    if table_name.upper() =='ASSIGNMENTS':
        db_cur = db_conn.cursor()
        db_cur.execute('SELECT * FROM track_log')
        t = db_cur.fetchone()
        for i in range(6,0,-1):
            df[t[i]] = df[t[i-1]] - df[t[i]]
        df[t[0]] = df['attempts'] - df[t[0]]
    return df

In [7]:
def get_pd_table(table_name, db_conn, student_id=""):
    
    if student_id != "":
        if table_name.upper() =='PROGRESS':
            df = pd.read_sql(f'SELECT * FROM {table_name} WHERE id = "{student_id}"', db_conn)
        else:
            df = pd.read_sql(f'SELECT * FROM {table_name} WHERE student_id = "{student_id}"', db_conn)
    else:
        df = pd.read_sql(f'SELECT * FROM {table_name}', db_conn)
        
    if table_name.upper() =='ASSIGNMENTS':
        db_cur = db_conn.cursor()
        db_cur.execute('SELECT * FROM track_log')
        t = db_cur.fetchone()
        dfcpy = df[[col for col in t]]
        for i in range(6):
            if dfcpy[t[i+1]].dtype == type(None):
                df[t[i]] = dfcpy[t[i]]
                break
            df[t[i]] = dfcpy[t[i]] - dfcpy[t[i+1]]
        asum =  df[[col for col in t]].sum(axis=1)
        assert (asum == df.attempts).astype(int).sum() == len(df), "Error: Sum of past attempts in not equal to total attempts" 
#         df.fillna(0, inplace=True)
    return df

In [12]:
import numpy as np
from random import randint
files = os.listdir('CSV_files/assignments')
for file in tqdm(files):
    df_asgn = pd.read_csv(f'CSV_files/assignments/{file}',encoding='utf-8')
#     print(file.split('-')[1].split('@')[0])
    df_asgn['Attempts']=0
    df_asgn['Attempts'] = df_asgn['Attempts'].apply(lambda x: x+ randint(0,6))
    update_assignment(file.split('-')[1].split('@')[0],df_asgn,'Khan', db_conn)


100%|██████████████████████████████████████████| 62/62 [00:13<00:00,  4.71it/s]


In [19]:
def change_attempts(db_conn, student_id='shaikhabdullah'):
    attempts = input('Enter the attempts: ')
    cur = db_conn.cursor()
    cur.execute('UPDATE Assignments SET attempts = ? WHERE student_id = ? AND assignment="Power"',(attempts, student_id))
    db_conn.commit()

In [26]:
change_attempts(db_conn)

Enter the attempts: 7


In [30]:
d = pd.read_sql('SELECT * FROM Assignments WHERE student_id = "shaikhabdullah" AND assignment="Power"',db_conn)
d

,assignment,student_id,teacher_id,due_date,status,attempts,best_score,"Sep 30,2020 at 20:08","Sep 30,2020 at 20:06","Sep 30,2020 at 20:04",last_1,last_2,last_3,last_4
0,Power,shaikhabdullah,Khan,"Jun 24th, 11:59 PM",Completed,7,-,7,6,5,None,None,None,None


In [31]:
df = get_pd_table('Assignments', db_conn, student_id='shaikhabdullah')
df.loc[df['assignment']=='Power']

,assignment,student_id,teacher_id,due_date,status,attempts,best_score,"Sep 30,2020 at 20:08","Sep 30,2020 at 20:06","Sep 30,2020 at 20:04",last_1,last_2,last_3,last_4
24,Power,shaikhabdullah,Khan,"Jun 24th, 11:59 PM",Completed,7,-,1,1,5,0,0,0,0


In [29]:
update_asgn_track(db_conn)

In [141]:
db_cur = db_conn.cursor()
db_cur.execute('SELECT * FROM track_log')
t = db_cur.fetchone()
asum =  df[[col for col in t]].sum(axis=1)
asum == df['attempts']
assert (asum == df.attempts).astype(int).sum() == len(df), "Error: Sum of past attempts in not equal to total attempts"

AssertionError: Error: Sum of past attempts in not equal to total attempts

In [22]:
db_cur =db_conn.cursor()
db_cur.execute('select * from track_log')
r = db_cur.fetchone()
r

('Sep 27,2020 at 17:28',
 'New',
 'Sep 27,2020 at 17:27',
 'last_1',
 'last_2',
 'last_3',
 'last_4')